### Imports

In [1]:
from tqdm import tqdm
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_mtsim
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
from gym_mtsim_forked.gym_mtsim.data import FOREX_DATA_PATH, FOREX_DATA_PATH_1HR, FOREX_DATA_PATH_15MIN, FOREX_DATA_PATH_5MIN
from gym_mtsim import OrderType, Timeframe, MtEnv, MtSimulator
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, STATUS_FAIL
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import time
import torch
import pickle
import fx_rl
from datetime import datetime, timedelta
import pytz

c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-unhedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-unhedge-v0 alr

## Latest Data

In [2]:
# now = datetime.now()
# current_time = pd.to_datetime(now) + timedelta(hours=7)
# current_time = current_time.replace(tzinfo=pytz.UTC)
# date_15min = fx_rl.bars_back(current_time, 'M15', total_bars=)
# date_5min = fx_rl.bars_back(current_time, 'M5')
# date_1hr = fx_rl.bars_back(current_time, 'H1')


In [3]:
# sim = MtSimulator(
#     unit='USD',
#     balance=10000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=False,
# )
# sim.download_data(
#     symbols=['EURUSD', 'AUDCHF', 'NZDCHF', 'GBPNZD', 'USDCAD'],
#     time_range=(
#         date_15min,
#         current_time
#     ),
#     timeframe=Timeframe.M15
# )
# sim.save_symbols(f'symbols_forex_15min_{date_15min.date()}_{current_time.date()}.pkl')

In [4]:
with open(f'symbols_forex_5min_2023-01-09_2024-05-08', 'rb') as f:
    symbols_5min = pickle.load(f)
# convert symbols_5min to a pd.dataframe
symbols_5min[1]['EURUSD'].index = pd.to_datetime(symbols_5min[1]['EURUSD'].index)
max_date_5min = symbols_5min[1]['EURUSD'].index.max()

In [5]:
with open(f'symbols_forex_15min_2020-05-11_2024-05-08', 'rb') as f:
    symbols_15min = pickle.load(f)
# convert symbols_15min to a pd.dataframe
symbols_15min[1]['EURUSD'].index = pd.to_datetime(symbols_15min[1]['EURUSD'].index)
max_date_15min = symbols_15min[1]['EURUSD'].index.max()

In [6]:
with open(f'symbols_forex_1hr_2008-05-19_2024-05-08', 'rb') as f:
    symbols_1hr = pickle.load(f)
# convert symbols_1hr to a pd.dataframe
symbols_1hr[1]['EURUSD'].index = pd.to_datetime(symbols_1hr[1]['EURUSD'].index)
# filter the symbols_15min[1]['EURUSD'] to where the index is greater than 2024-3-24
one_month_training = symbols_1hr[1]['EURUSD'][symbols_1hr[1]['EURUSD'].index > '2024-01-01']
max_date_1hr = one_month_training.index.max()
max_date_1hr

Timestamp('2024-05-08 13:00:00+0000', tz='UTC')

In [7]:
# # unpack the pickle file and load the data that is in symbols_forex.pkl
# with open(FOREX_DATA_PATH, 'rb') as f:
#     symbols = pickle.load(f)
# # convert symbols to a pd.dataframe
# # symbols[1]['EURUSD']

In [8]:
slices_5min = fx_rl.slices_finder(symbols_5min[1]['EURUSD'], max_date_5min, testing_needed=True)
slices_15min = fx_rl.slices_finder(symbols_15min[1]['EURUSD'], max_date_15min, testing_needed=True)
slices_1hr = fx_rl.slices_finder(symbols_1hr[1]['EURUSD'], max_date_1hr, testing_needed=True)

In [9]:
slices_15min[0]

DatetimeIndex(['2020-05-11 21:30:00+00:00', '2020-05-11 21:45:00+00:00',
               '2020-05-11 22:00:00+00:00', '2020-05-11 22:15:00+00:00',
               '2020-05-11 22:30:00+00:00', '2020-05-11 22:45:00+00:00',
               '2020-05-11 23:00:00+00:00', '2020-05-11 23:15:00+00:00',
               '2020-05-11 23:30:00+00:00', '2020-05-11 23:45:00+00:00',
               ...
               '2024-04-19 21:30:00+00:00', '2024-04-19 21:45:00+00:00',
               '2024-04-19 22:00:00+00:00', '2024-04-19 22:15:00+00:00',
               '2024-04-19 22:30:00+00:00', '2024-04-19 22:45:00+00:00',
               '2024-04-19 23:00:00+00:00', '2024-04-19 23:15:00+00:00',
               '2024-04-19 23:30:00+00:00', '2024-04-19 23:45:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Time', length=98240, freq=None)

In [10]:
slices_1hr[0]

DatetimeIndex(['2008-05-19 21:00:00+00:00', '2008-05-19 22:00:00+00:00',
               '2008-05-19 23:00:00+00:00', '2008-05-20 00:00:00+00:00',
               '2008-05-20 01:00:00+00:00', '2008-05-20 02:00:00+00:00',
               '2008-05-20 03:00:00+00:00', '2008-05-20 04:00:00+00:00',
               '2008-05-20 05:00:00+00:00', '2008-05-20 06:00:00+00:00',
               ...
               '2024-04-19 14:00:00+00:00', '2024-04-19 15:00:00+00:00',
               '2024-04-19 16:00:00+00:00', '2024-04-19 17:00:00+00:00',
               '2024-04-19 18:00:00+00:00', '2024-04-19 19:00:00+00:00',
               '2024-04-19 20:00:00+00:00', '2024-04-19 21:00:00+00:00',
               '2024-04-19 22:00:00+00:00', '2024-04-19 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Time', length=98716, freq=None)

### Create Env

In [11]:
# class MyMtEnv(gym_mtsim.MtEnv):
#     # _get_modified_volume = fx_rl.my_get_modified_volume
#     _get_prices = fx_rl.my_get_prices

In [12]:
def env_fx(timeframe, slices):
    if timeframe == '1hr':
        data_path = FOREX_DATA_PATH_1HR
    elif timeframe == '15min':
        data_path = FOREX_DATA_PATH_15MIN
    elif timeframe == '5min':
        data_path = FOREX_DATA_PATH_5MIN
    sim_training = gym_mtsim.MtSimulator(
        unit='USD',
        balance=200000.,
        leverage=100.,
        stop_out_level=0.2,
        hedge=True,
        symbols_filename=data_path
    )
    env_training = gym_mtsim.MtEnv(
        original_simulator=sim_training,
        trading_symbols=['EURUSD'],
        window_size = 10,
        time_points= slices[0],
        hold_threshold=0.5,
        close_threshold=0.5,
        fee=lambda symbol: {
            # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
            'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
            # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
        }[symbol],
        symbol_max_orders=2,
        multiprocessing_processes=2
    )
    sim_validation = gym_mtsim.MtSimulator(
        unit='USD',
        balance=200000.,
        leverage=100.,
        stop_out_level=0.2,
        hedge=True,
        symbols_filename=data_path
    )
    env_validation = gym_mtsim.MtEnv(
        original_simulator=sim_validation,
        trading_symbols=['EURUSD'],
        window_size = 10,
        time_points= slices[1],
        hold_threshold=0.5,
        close_threshold=0.5,
        fee=lambda symbol: {
            # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
            'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
            # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
        }[symbol],
        symbol_max_orders=2,
        multiprocessing_processes=2
    )
    return env_training, env_validation


In [13]:
# env_validation = MyMtEnv(
#     original_simulator=sim_train_1hr,
#     trading_symbols=['EURUSD'],
#     window_size = 10,
#     time_points=slices_1hr[1],
#     hold_threshold=0.5,
#     close_threshold=0.5,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

In [14]:
# sim_testing = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH_1HR
# )

# env_testing = MyMtEnv(
#     original_simulator=sim_testing,
#     trading_symbols=['EURUSD'],
#     window_size = 10,
#     time_points=slices_1hr[2],
#     hold_threshold=0.5,
#     close_threshold=0.5,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

### Define Functions

In [15]:
def print_stats(reward_over_episodes, printing_name):
    """  Print Reward  """

    avg_rewards = np.mean(reward_over_episodes)
    min_rewards = np.min(reward_over_episodes)
    max_rewards = np.max(reward_over_episodes)

    print (f'Min. {printing_name}          : {min_rewards:>10.3f}')
    print (f'Avg. {printing_name}          : {avg_rewards:>10.3f}')
    print (f'Max. {printing_name}          : {max_rewards:>10.3f}')

    return min_rewards, avg_rewards, max_rewards

def my_profit_calculation(env_orders, stop_loss):
        # env_orders = env_testing.render()['orders']
        # stop_loss = 0.001
        mean_value = env_orders['Volume'].mean()

        # # Normalize the column to have a mean of 1
        env_orders.loc[:, 'Volume'] = round((env_orders['Volume'] / mean_value), 2)

        # add a column for when the difference between the Entry Price and the Exit Price is greater than stop_loss
        env_orders.loc[:, 'stoploss_hit'] = np.where((env_orders['Type'].str.strip() == 'Buy') &
                                                        ((env_orders['Entry Price'] - env_orders['Exit Price']) > stop_loss),
                                                        1, np.where((env_orders['Type'].str.strip() == 'Sell') &
                                                                        ((env_orders['Exit Price'] - env_orders['Entry Price']) > stop_loss),
                                                                        1, 0))
        env_orders.loc[:, 'Exit Price'] = np.where((env_orders['Type'].str.strip() == 'Buy') & (env_orders['stoploss_hit'] == 1),
                                                        env_orders['Entry Price'] - stop_loss,
                                                        np.where((env_orders['Type'].str.strip() == 'Sell') & (env_orders['stoploss_hit'] == 1),
                                                                env_orders['Entry Price'] + stop_loss, env_orders['Exit Price']))
        env_orders.loc[:, 'Profit'] = np.where((env_orders['Type'].str.strip() == 'Buy'),
                                                        ((env_orders['Exit Price'] - (env_orders['Fee']/2)) - 
                                                        (env_orders['Entry Price'] + (env_orders['Fee']/2)))
                                                                * 100_000 * env_orders['Volume'], 
                                                        np.where((env_orders['Type'].str.strip() == 'Sell'),
                                                                ((env_orders['Entry Price'] - (env_orders['Fee']/2)) - 
                                                                (env_orders['Exit Price'] + (env_orders['Fee']/2)))
                                                                * 100_000 * env_orders['Volume'], np.nan))
        total_reward = env_orders.loc[:, 'Profit'].sum()
        # Calculate Gross Profit
        gross_profit = env_orders.loc[env_orders['Profit'] > 0, 'Profit'].sum()

        # Calculate Gross Loss
        gross_loss = env_orders.loc[env_orders['Profit'] < 0, 'Profit'].abs().sum()

        # Calculate Profit Factor
        profit_factor = gross_profit / gross_loss if gross_loss != 0 else 0

        profit_factor = profit_factor - 1

        return profit_factor, total_reward

# ProgressBarCallback for model.learn()
class ProgressBarCallback(BaseCallback):

    def __init__(self, check_freq: int, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.progress_bar = tqdm(total=self.model._total_timesteps, desc="model.learn()")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            self.progress_bar.update(self.check_freq)
        return True
    
    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.progress_bar.close()


In [16]:
space = {
    # 'learning_rate': hp.loguniform('learning_rate', -5, -2), # Learning rate
    'learning_rate': hp.uniform('learning_rate', 0, 0.08), # Learning rate
    'gamma': hp.uniform('gamma', 0.925, 0.975), # Discount factor
    # 'ent_coef': hp.loguniform('ent_coef', -5, 0) # Entropy coefficient
    'ent_coef': hp.uniform('ent_coef', 0, 0.2), # Entropy coefficient
    # 'learning_timesteps': hp.choice('learning_timesteps', [25, 50, 100, 250, 500]),
    'timeframe': hp.choice('timeframe', ['5min', '15min', '1hr']),
    # 'n_bars': hp.uniform('nbars', 24, 100_000)
}

In [17]:
day_max_15min_slices = [96, 192, 288, 384, 480]
day_max_1hr_slices = [int(i/4) for i in day_max_15min_slices]
day_max_5min_slices = [int(i*12) for i in day_max_1hr_slices]

min_15min_slices = [0, 96, 192, 288, 384]
min_1hr_slices = [int(i/4) for i in min_15min_slices]
min_5min_slices = [int(i*12) for i in min_1hr_slices]

print(day_max_15min_slices, day_max_1hr_slices, day_max_5min_slices)
print(min_15min_slices, min_1hr_slices, min_5min_slices)

[96, 192, 288, 384, 480] [24, 48, 72, 96, 120] [288, 576, 864, 1152, 1440]
[0, 96, 192, 288, 384] [0, 24, 48, 72, 96] [0, 288, 576, 864, 1152]


In [18]:
# TRAINING + TEST
def train_val_model(model, model_policy, seed, steps_str, lr, gamma_param, entropy, timeframe, total_learning_timesteps=10_000):#env_tr, env_val,
    """
    Trains and validates a model using the Proximal Policy Optimization (PPO) algorithm.

    Args:
        model (object): The model to be trained.
        model_policy (object): The policy used by the model.
        env_tr (object): The training environment.
        env_val (object): The validation environment.
        seed (int): The random seed for reproducibility.
        steps_str (str): A string representing the number of steps.
        window_size_param (int): The window size parameter.
        lr (float): The learning rate.
        gamma_param (float): The gamma parameter.
        entropy (float): The entropy coefficient.
        total_learning_timesteps (int, optional): The total number of learning timesteps. Defaults to 10,000.

    Returns:
        tuple: A tuple containing the reward over validations, orders over validations, and the model dictionary.
    """
    # reproduce training and test
    print('-' * 80)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    num_episodes = 10
    reward_over_tests = {}
    orders_over_tests = {}
    profit_over_tests = {}
    if timeframe == '5min':
        min_slices, day_max_slices = [0, 288, 576, 864, 1152], [288, 576, 864, 1152, 1440]
        slices = slices_5min
    elif timeframe == '15min':
        min_slices, day_max_slices = [0, 96, 192, 288, 384], [96, 192, 288, 384, 480]
        slices = slices_15min
    elif timeframe == '1hr':
        min_slices, day_max_slices = [0, 24, 48, 72, 96], [24, 48, 72, 96, 120]
        slices = slices_1hr
    
    for mins, maxes in zip(min_slices, day_max_slices):
        slices_ready = [0, 0]
        # print(f'Using {mins} of type {type(mins)} to {maxes} of type{type(maxes)} slices')
        slices_ready[1] = slices[1][mins:maxes]
        timestep_max = slices_ready[1][0]
        slices_tmp = slices[1][slices[1] < timestep_max]
        slices_ready[0] = slices[0].append(slices_tmp)
        
        env_tr, env_val = env_fx(timeframe, slices_ready)
        obs_val, info_val = env_val.reset(seed=seed)
        obs_tr, info_tr = env_tr.reset(seed=seed)
        model = PPO(model_policy, env_tr, verbose=0, ent_coef=entropy, learning_rate=lr, gamma=gamma_param)
        
        # custom callback for 'progress_bar'
        model.learn(total_timesteps=total_learning_timesteps)#, callback=ProgressBarCallback(100))
        reward_over_validations = []
        orders_over_validations = []
        profit_over_validations = []



        for episode in range(num_episodes):
            obs_val, info_val = env_val.reset(seed=seed)

            total_reward = 0
            done_val = False

            while not done_val:
                action, _states = model.predict(obs_val)
                obs_val, reward_val, terminated_val, truncated_val, info_val = env_val.step(action)
                done_val = terminated_val or truncated_val

                total_reward += reward_val
                if done_val:
                    break
            try:
                orders_made_in_episode = env_val.render()['orders']
                order_len = len(orders_made_in_episode)
                total_reward, total_profit = my_profit_calculation(orders_made_in_episode, 0.001)
                
            except Exception as e:
                print(f'There were not any orders produced by the model. Error: {e}')
                order_len = 0
                total_profit = 0

            # model_dict[f'model_{episode}'] = model
            # model.save(f'best_hyperparameters/models_4_26_24/model_{episode}.pkl')

            reward_over_validations.append(total_reward) 
            profit_over_validations.append(total_profit)   
            orders_over_validations.append(order_len)  


            # if episode % 1 == 0:
            avg_reward = np.mean(reward_over_validations)
            avg_orders = np.mean(orders_over_validations)
            avg_profit = np.mean(profit_over_validations)
            # print(f'Episode: {episode}, Avg. Reward: {avg_reward:.3f}, # of orders: {avg_orders:.3f}, avg Profit: {avg_profit:.3f}')
        reward_over_tests[f'{mins}'] = reward_over_validations
        orders_over_tests[f'{mins}'] = orders_over_validations
        profit_over_tests[f'{mins}'] = profit_over_validations
        min_profit_day, avg_profit_day, max_profit_day = print_stats(profit_over_validations, 'Profit')
        min_orders_day, avg_orders_day, max_orders_day = print_stats(orders_over_validations, 'Orders')
        print('-' * 80)
        # get the average reward, orders, and profit for the week by finding the mean of the reward_over_tests, orders_over_tests, and profit_over_tests dictionaries
    avg_reward_week = np.sum([np.sum(reward_over_tests[key]) for key in reward_over_tests.keys()])/(num_episodes*len(reward_over_tests.keys()))
    avg_orders_week = np.sum([np.sum(orders_over_tests[key]) for key in orders_over_tests.keys()])/(num_episodes*len(orders_over_tests.keys()))
    avg_profit_week = np.sum([np.sum(profit_over_tests[key]) for key in profit_over_tests.keys()])/(num_episodes*len(profit_over_tests.keys()))
    min_profit_week = np.sum([np.min(profit_over_tests[key]) for key in profit_over_tests.keys()])
    max_profit_week = np.sum([np.max(profit_over_tests[key]) for key in profit_over_tests.keys()])

    print(f'Avg Profit for Week: {avg_profit_week:.3f}, Min Profit for Week: {min_profit_week:.3f}, Max Profit for Week: {max_profit_week:.3f}')

    return avg_reward_week, avg_orders_week, avg_profit_week#, model_dict

## Objective Function

In [19]:
seed = 2024  # random seed
total_num_episodes = 10

# print ("env_name                 :", env_name)
print ("seed                     :", seed)

# INIT matplotlib
plot_settings = {}
plot_data = {'x': [i for i in range(1, total_num_episodes + 1)]}

# learning_timesteps_list_in_K = [25]#, 50, 100]
# learning_timesteps_list_in_K = [50, 250, 500]
# learning_timesteps_list_in_K = [500, 1000, 3000, 5000]

# RL Algorithms: https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

timesteps_models_dict = {}
def objective(params):
    learning_timesteps = 100 #params['learning_timesteps']
    ent_coef = params['ent_coef']#0#
    gamma = params['gamma'] #0.99 #
    learning_rate = params['learning_rate']#0.0003#
    # bars = params['n_bars']
    timeframe = params['timeframe']
    print(f'----- timeframe: {timeframe} ------')
    total_learning_timesteps = learning_timesteps * 1000
    step_key = f'{learning_timesteps}K'
    policy_dict = PPO.policy_aliases
    policy = policy_dict.get('MultiInputPolicy')
    class_name = type(PPO).__qualname__
    plot_key = f'{class_name}_rewards_'+step_key
    try:
        avg_rewards, avg_orders, avg_profits = train_val_model(PPO, policy, seed, step_key,  
                                                    learning_rate, gamma, ent_coef, timeframe, total_learning_timesteps)
    except Exception as e:
        if 'Tensor of shape' in str(e):
            print(f'''there was an error with the tensor with those parameters: \n
              ent_coef: {ent_coef}, gamma: {gamma}, learning_rate: {learning_rate}''')
        else:
            print(f'''there was an error {e} with those parameters: timesteps: {learning_timesteps}, \n
                ent_coef: {ent_coef}, gamma: {gamma}, learning_rate: {learning_rate}''')
        return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
    # timesteps_models_dict[step_key] = models_dict
    # min_rewards, avg_rewards, max_rewards, = print_stats(rewards, 'Reward')
    # print_stats(orders, 'Orders')
    # label = f'Avg. {avg_rewards:>7.2f} : {class_name} - {step_key}'
    # plot_data[plot_key] = rewards
    # plot_settings[plot_key] = {'label': label}
    params['avg_orders'] = avg_orders
    params['profits'] = avg_rewards       

    return {'loss': -avg_profits, 'status': STATUS_OK, 'eval_time': time.time(), 'parameters': params} 

seed                     : 2024


In [20]:
# avg_rewards, avg_orders, avg_profits = train_val_model(PPO, 'MultiInputPolicy', seed, '100K',  
#                                             0.002722590750866436, 0.9523416085548663, 0.047694108939137815, '5min', 100_000)

In [21]:
#### adding in gamma test ####
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=200, # Number of evaluations of the objective function
            trials=trials,
            trials_save_file=f'gym_mtsim_forked/examples/hyperopt/trials_5_16_timeframe_analysis_daily_retraining.pkl')

print("Best parameters:", best)

----- timeframe: 1hr ------                            
--------------------------------------------------------------------------------
there was an error with the tensor with those parameters: 

              ent_coef: 0.046765605339684505, gamma: 0.951821023693376, learning_rate: 0.07819475077030365
----- timeframe: 1hr ------                                      
--------------------------------------------------------------------------------
Min. Profit          :   -635.592                                
Avg. Profit          :    -80.156                                
Max. Profit          :    559.648                                
Min. Orders          :      7.000                                
Avg. Orders          :      8.500                                
Max. Orders          :     10.000                                
Min. Profit          :   -205.635                                
Avg. Profit          :     80.474                                
Max. Profit          